In [32]:
!pip install findspark
!pip install contractions
!pip install num2words
!pip install nltk
!pip install pandas
!pip install pyarrow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 92.4 MB/s eta 0:00:0031m101.9 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default"
os.environ["SPARK_HOME"] = "spark-3.5.1-bin-hadoop3"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/21 22:37:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/21 22:37:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Preprocessing

## Import Data

In [6]:
df = spark.read.csv('data/tweets.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- Target: integer (nullable = true)
 |-- ID: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- User: string (nullable = true)
 |-- Text: string (nullable = true)



In [7]:
df_orig = df

df_orig.show(5, truncate=False)

+------+----------+----------------------------+--------+-------------+---------------------------------------------------------------------------------------------------------------+
|Target|ID        |Date                        |flag    |User         |Text                                                                                                           |
+------+----------+----------------------------+--------+-------------+---------------------------------------------------------------------------------------------------------------+
|0     |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY|scotthamilton|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!|
|0     |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO_QUERY|mattycus     |@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                      |
|0     |1467811184|Mon Apr 06 22:19:57 PDT 2009|NO_QUERY|ElleCTF      |my whole 

## Dataframe Preprocessing

In [9]:
from pyspark.sql.functions import to_timestamp, weekday, hour, col, when
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

df = df.drop('flag')

datetime_string = "EEE MMM dd HH:mm:ss z yyyy"
df = df.withColumn('Date', to_timestamp(df.Date, datetime_string))
df = df.withColumn('Day', weekday(df.Date))
df = df.withColumn('Hour', hour(df.Date))

df = df.withColumnRenamed('Target', 'Positive')
df = df.withColumn('Positive', when(col('Positive') == 4, 1).otherwise(0))

df.show(5)

+--------+----------+-------------------+-------------+--------------------+---+----+
|Positive|        ID|               Date|         User|                Text|Day|Hour|
+--------+----------+-------------------+-------------+--------------------+---+----+
|       0|1467810672|2009-04-07 00:19:49|scotthamilton|is upset that he ...|  1|   0|
|       0|1467810917|2009-04-07 00:19:53|     mattycus|@Kenichan I dived...|  1|   0|
|       0|1467811184|2009-04-07 00:19:57|      ElleCTF|my whole body fee...|  1|   0|
|       0|1467811193|2009-04-07 00:19:57|       Karoli|@nationwideclass ...|  1|   0|
|       0|1467811372|2009-04-07 00:20:00|     joy_wolf|@Kwesidei not the...|  1|   0|
+--------+----------+-------------------+-------------+--------------------+---+----+
only showing top 5 rows



In [10]:
crime_keywords = ['murder', 'murderer', 'theft', 'assault', 'robbery', 'stole', 'steal', 'stealing', 'burglary', 'arrest', 'arson', 'drug', 'drugs', 'police', 'suspect', 'violence']

df = df.withColumn(
    'crime_related',
    when(col('Text').rlike("(?i)\\b(" + "|".join(crime_keywords) + ")\\b"), 1).otherwise(0)
)

df.groupBy('crime_related').count().show()

+-------------+-------+
|crime_related|  count|
+-------------+-------+
|            1|   2225|
|            0|1046350|
+-------------+-------+



In [11]:
df_crime = df.filter(col('crime_related')==1)
df_crime.select('Text').show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------+
|Text                                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------------------------------+
| Body Of Missing Northern Calif. Girl Found: Police have found the remains of a missing Northern California girl .. http://tr.im/imji|
|augh, eff sarth  i stole some of MIL's nail polish, its pretty! =o                                                                   |
|so a murder? gotcha.  Cant believe it                                                                                                |
|Megan equals Murder.                                                                                                                 |
|@grahamcracker  If only you were working in the

In [12]:
from pyspark.sql.functions import rand, floor

# simulate zip code approximation from IP address
df = df.withColumn("Zip", floor(rand() * 70).cast("int"))

df.show(5)

+--------+----------+-------------------+-------------+--------------------+---+----+-------------+---+
|Positive|        ID|               Date|         User|                Text|Day|Hour|crime_related|Zip|
+--------+----------+-------------------+-------------+--------------------+---+----+-------------+---+
|       0|1467810672|2009-04-07 00:19:49|scotthamilton|is upset that he ...|  1|   0|            0| 25|
|       0|1467810917|2009-04-07 00:19:53|     mattycus|@Kenichan I dived...|  1|   0|            0| 23|
|       0|1467811184|2009-04-07 00:19:57|      ElleCTF|my whole body fee...|  1|   0|            0| 31|
|       0|1467811193|2009-04-07 00:19:57|       Karoli|@nationwideclass ...|  1|   0|            0| 58|
|       0|1467811372|2009-04-07 00:20:00|     joy_wolf|@Kwesidei not the...|  1|   0|            0| 24|
+--------+----------+-------------------+-------------+--------------------+---+----+-------------+---+
only showing top 5 rows



In [24]:
count_0 = df.filter(df.crime_related == 0).count()
count_1 = df.filter(df.crime_related == 1).count()

class_ratio = count_1 / count_0

df_0 = df.filter(df.crime_related == 0).sample(withReplacement=False, fraction=class_ratio, seed=42)
df_1 = df.filter(df.crime_related == 1)

df_balanced = df_0.union(df_1)

# shuffle rows
df_balanced = df_balanced.orderBy(rand())

df_balanced.show()

[Stage 52:============================================>           (32 + 8) / 40]

+--------+----------+-------------------+--------------+--------------------+---+----+-------------+---+
|Positive|        ID|               Date|          User|                Text|Day|Hour|crime_related|Zip|
+--------+----------+-------------------+--------------+--------------------+---+----+-------------+---+
|       0|1556829516|2009-04-19 02:33:07|   bumsonseats|@needanewone poli...|  6|   2|            1| 56|
|       0|1989513537|2009-06-01 03:49:36|    ryanbeales|Crazy city homele...|  0|   3|            1|  3|
|       0|2211485008|2009-06-17 14:54:48|         McZoe|I could murder a ...|  2|  14|            1|  8|
|       0|1679336817|2009-05-02 11:06:38|       nabeelq|Landon keeps stea...|  5|  11|            1| 55|
|       0|2178537770|2009-06-15 09:50:51|       brajana|OMG Noooooo!!!!  ...|  0|   9|            1| 50|
|       0|2215072217|2009-06-17 19:38:07|         xxmah|@viic_x oh, i won...|  2|  19|            0| 22|
|       0|1688896931|2009-05-03 14:10:02|        rubyku

## Text Preprocessing

In [39]:
import re
import contractions
import nltk
import pyarrow
import html

import pandas as pd

from num2words import num2words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('universal_tagset')

[nltk_data] Downloading package wordnet to /home/antlers/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antlers/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/antlers/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [18]:
acronyms = {
    "mr.": "mister", "mrs.": "misses", "dr.": "doctor", "st.": "street",
    "u.s.": "united states", "e.g.": "for example", "i.e.": "that is",
    "vs.": "versus", "w/": "with", "w/o": "without", "n/a": "not applicable",
    "thx": "thanks", "u": "you", "wut": "what", "wtf": "what the fuck",
    "idk": "i do not know", "luv": "love", "irl": "in real life"
}
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return None

In [42]:
### Preprocessing Functions
def expand_contractions_and_acronyms(text):
    text = contractions.fix(text)
    pattern = re.compile(r'\b(' + '|'.join(acronyms.keys()) + r')\b', re.IGNORECASE)
    text = pattern.sub(lambda m: acronyms.get(m.group(0).lower(), m.group(0)), text)
    return text

def handle_web_elements(text):
    text = re.sub(r'@[\w_]+', '<user>', text)
    text = re.sub(r'#[\w_]+', '<hashtag>', text)
    text = re.sub(r'\$', ' dollar ', text)
    text = re.sub(r'€', ' euro ', text)
    text = re.sub(r'http\S+', 'URL', text)
    return text

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

def lemmatize_with_pos(tokens):
    pos_tags = pos_tag(tokens)
    lemmas = []
    for token, tag in pos_tags:
        wn_tag = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(token, pos=wn_tag) if wn_tag else lemmatizer.lemmatize(token)
        lemmas.append(lemma)
    return lemmas

def preprocess_text(text, use_spelling_correction=False, convert_numbers=False):
    text = text.lower()
    text = html.unescape(text)
    text = expand_contractions_and_acronyms(text)
    text = handle_web_elements(text)
    text = re.sub(r'[+-]?(\d*\.)?\d+', lambda m: num2words(m.group()), text)
    text = remove_punctuation(text)
    tokens = word_tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize_with_pos(tokens)
    return ' '.join(tokens)

In [41]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType

@pandas_udf(StringType())
def preprocess_text_udf(text_series):
    return text_series.apply(preprocess_text)

df_balanced = df_balanced.withColumn("Text_Preprocessed", preprocess_text_udf(df_balanced["Text"]))
df_balanced.show(5)

[Stage 72:>                                                         (0 + 1) / 1]

+--------+----------+-------------------+-----------+--------------------+---+----+-------------+---+--------------------+
|Positive|        ID|               Date|       User|                Text|Day|Hour|crime_related|Zip|   Text_Preprocessed|
+--------+----------+-------------------+-----------+--------------------+---+----+-------------+---+--------------------+
|       0|1556829516|2009-04-19 02:33:07|bumsonseats|@needanewone poli...|  6|   2|            1| 56|user police dog c...|
|       0|1989513537|2009-06-01 03:49:36| ryanbeales|Crazy city homele...|  0|   3|            1|  3|crazy city homele...|
|       0|2211485008|2009-06-17 14:54:48|      McZoe|I could murder a ...|  2|  14|            1|  8|could murder cold...|
|       0|1679336817|2009-05-02 11:06:38|    nabeelq|Landon keeps stea...|  5|  11|            1| 55|landon keep steal...|
|       0|2178537770|2009-06-15 09:50:51|    brajana|OMG Noooooo!!!!  ...|  0|   9|            1| 50|omg noooooo stupi...|
+--------+------

# Modeling

## Text Vectorization

## Training Split